In [ ]:
import cx_Oracle
import getpass
import pandas as pd
import numpy as np

In [ ]:
bruker = input('Angi bruker: ') or getpass.getuser()
bruker

In [ ]:
#getpass.getpass(prompt='Angi oracle-passord for bruker ' + bruker + ':')

In [ ]:
conn = cx_Oracle.connect(bruker + "/" + getpass.getpass(prompt='Angi oracle-passord for bruker ' + bruker + ':')+"@DB1P")


In [ ]:
#Denne ser ut til å fungere ganske bra(?)

query = """
 select *
 from dsbbase.ssb_foretak
 where (sektor_2014 = '6500' or sektor_2014 = '1510') and  statuskode = 'B' and org_form = 'IKS'
 """

iks = pd.read_sql_query(query, conn)
iks = iks[['ORGNR','ORG_FORM','STATUSKODE', 'SEKTOR_2014', 'UNDERSEKTOR_2014', 'NAVN', 'F_KOMMUNENR']]
iks
#, 'status', 'sektor_2014', 'undersektor_2014', 'navn', 'fkommune', 'statdat', 'sektor2014_dato'
#statdato må enten være STATUSKODE_GDATO eller STATUSKODE_RDATO
#sektor2014_dato må enten være SEKTOR_2014_GDATO ELLER SEKTOR_2014_RDATO

In [ ]:
query = """
 select *
 from dsbbase.ssb_foretak
 where (org_form = 'KOMM' or org_form = 'FYLK' or org_form = 'IKS') and  (statuskode = 'B' or statuskode = 'S')
 """

komm = pd.read_sql_query(query, conn)

komm = komm[['ORGNR','ORG_FORM','STATUSKODE', 'NAVN', 'F_KOMMUNENR']]

komm['sortering']= np.where(komm['ORG_FORM']== 'KOMM', 1, 2)

komm.rename(columns={'ORGNR':'orgnr_eier','NAVN':'navn_eier', 'F_KOMMUNENR':'kommnr_eier', 'STATUSKODE':'status_eier', 'ORG_FORM':'orgform_eier'}, inplace = True)

komm

In [ ]:
conn.close()

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
komm

In [ ]:
kommuner = pd.read_parquet("../../../../Testdata/arbeidsdata/klass/kommuner.parquet")
fykomm = pd.read_parquet("../../../../Testdata/arbeidsdata/klass/fykomm.parquet")
regioner = pd.concat([kommuner, fykomm]).rename(columns={'region':'kommnr_eier'})

In [ ]:
gyldige=pd.merge(komm, regioner, on=['kommnr_eier'])
gyldige

In [ ]:
uoppgitt = komm.merge(regioner,indicator = True, how='left').loc[lambda x : x['_merge']!='both']
uoppgitt